Will need to install package from: https://github.com/tiagocoutinho/hamamatsu/tree/master

In [ ]:
from hamamatsu.dcam import dcam, Stream, copy_frame
import logging

import matplotlib.pyplot as plt

In [ ]:
with dcam:
    camera = dcam[0]

In [ ]:
with camera:

    # Simple acquisition example
    nb_frames = 10
    camera["exposure_time"] = 0.1
    with Stream(camera, nb_frames) as stream:
            logging.info("start acquisition")
            camera.start()


In [ ]:
for i, f in enumerate(stream):
    print(f)

In [ ]:
with dcam:
    camera = dcam[0]
with camera:
    print(camera.info)
    print(camera['image_width'].value, camera['image_height'].value)

    # Simple acquisition example
    nb_frames = 10
    camera["exposure_time"] = 0.1
    with Stream(camera, nb_frames) as stream:
            logging.info("start acquisition")
            camera.start()
            for i, frame_buffer in enumerate(stream):
                frame = copy_frame(frame_buffer)
                # plt.figure()
                # plt.imshow(frame)
                logging.info(f"acquired frame #%d/%d: %s", i+1, nb_frames, frame)
            logging.info("finished acquisition")

In [ ]:
from hamamatsu.dcam import dcam, Stream, copy_frame
import logging

import matplotlib.pyplot as plt

from nidaqmx import Task
from nidaqmx.constants import AcquisitionType, Edge
from nidaqmx.stream_readers import AnalogMultiChannelReader
from nidaqmx.stream_writers import AnalogMultiChannelWriter

from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QHBoxLayout, QPushButton, QMessageBox, QFileDialog
from PyQt5 import QtWidgets, QtCore
from superqt import QLabeledDoubleRangeSlider, QLabeledDoubleSlider, QLabeledSlider
import pyqtgraph as pg

import numpy as np
import sys
from skimage import io

import flammkuchen as fl

import time

In [ ]:
class ConfocalMicroscopy(QWidget):
    def __init__(self, samp_rate=10000, amp=3, num_px=100):
        super().__init__()

        self.dev = "Dev4"
        self.ao_ch = "ao3" # cam_trigger

        self.rate = samp_rate # samples per channel per second
        self.amp = amp # scanning range in volt
        self.x_offset = 0
        self.y_offset = 0

        ######################################################
        # self.fps = 1
        # self.ao_write_rate = 100
        self.acq_dur = 1
        self.exposure_time = 0.1
        # self.n_samples = ao_write_rate*acq_dur
        #######################################################
        # self.timer = QtCore.QTimer(self)
        # self.timer.timeout.connect(self.display_frame)
        # self.timer.start(30)  # Update the frame every 30 milliseconds
        self.frames_buffer = []
        #########################################

        self.num_px = num_px # the number of pixels for both x and y
        self.total_px = self.num_px * self.num_px # aspect ratio = 1

        # self.write_signal = self.waveform()
        self.read_buffer = np.empty((1, self.total_px))
        self.buf_size = 5 # times bigger than one frame
        self.stack = []

        self.running = False
        self.first_img = True

        self.last_frame = []

        self.set_camera()
        self.set_gui()

    def set_camera(self):
        with dcam:
            self.camera = dcam[0]
        # self.camera["exposure_time"] = self.exposure_time
        
        print(self.camera)
        
    def set_gui(self):
        self.start_button = QPushButton("Start acquisition")
        self.start_button.clicked.connect(self.toggle)

        self.save_button = QPushButton("Save last acquisition")
        # self.save_button.clicked.connect(self.save_acquisition)

        self.x_amp = QLabeledDoubleSlider(QtCore.Qt.Horizontal)
        self.x_amp.setRange(0.1, 6)
        self.x_amp.setValue(3)
        self.setFocusPolicy(QtCore.Qt.NoFocus)
        # self.x_amp.valueChanged.connect(self.update_acq_params)

        # self.srate_slider = QLabeledDoubleSlider(QtCore.Qt.Horizontal)
        # self.srate_slider.setRange(self.total_px, self.total_px*10)
        # self.srate_slider.setSingleStep(self.total_px)
        # self.srate_slider.setValue(3)
        # self.setFocusPolicy(QtCore.Qt.NoFocus)
        # self.srate_slider.valueChanged.connect(self.update_acq_params)

        self.viewer = pg.ImageView()
        self.viewer.getHistogramWidget().setHistogramRange(0, 1)
        self.viewer.ui.roiBtn.hide()
        self.viewer.ui.menuBtn.hide()

        layout = QVBoxLayout()

        layout2 = QHBoxLayout()
        layout2.addWidget(self.start_button)
        layout2.addWidget(self.save_button)
        layout.addLayout(layout2)

        layout3 = QHBoxLayout()
        layout3.addWidget(QLabel("Zoom (V)"))
        layout3.addWidget(self.x_amp)
        layout.addLayout(layout3)

        layout4 = QHBoxLayout()
        layout4.addWidget(QLabel("Sampling rate (samples/s)"))
        layout4.addWidget(QLabel("{:.3f}fps".format(self.rate/self.total_px)))
        # layout4.addWidget(self.srate_slider)
        layout.addLayout(layout4)

        layout.addWidget(self.viewer)

        self.setLayout(layout)

        
    def toggle(self):
        if self.running:
            self.stop()
        else:
            self.start()

            
    def start(self):
        self.start_button.setText("Stop acquisition")
        self.start_button.setStyleSheet("background-color: yellow")
        self.running = True

        self.set_tasks()
        self.read_camera()
        
        #######################################################
        self.timer = QtCore.QTimer(self)
        self.timer.timeout.connect(self.display_frame)
        self.timer.start(30)  # Update the frame every 30 milliseconds
        # self.frames_buffer = []
        #########################################

        

        
    def read_camera(self):
        with self.camera:
            # Simple acquisition example
            nb_frames = 10
            self.camera["exposure_time"] = 1
            with Stream(self.camera, nb_frames) as stream:
                    logging.info("start acquisition")
                    self.camera.start()
                    for i, frame_buffer in enumerate(stream):
                        self.frames_buffer.append(copy_frame(frame_buffer))
                        print(len(self.frames_buffer))
                    #     logging.info(f"acquired frame #%d/%d: %s", i+1, nb_frames, frame)
                    # logging.info("finished acquisition")
                    
                    
    def stop(self):
        self.start_button.setText("Start acquisition")
        self.start_button.setStyleSheet("")
        self.running = False

        self.write_task.stop()
        self.camera.stop_acquisition()
        # fl.save(r'C:\Users\portugueslab\Desktop\smth.h5',self.stack )
        # self.write_task.stop()
        # self.write_task.close()
        # self.read_task.stop()
        # self.read_task.close()

        # self.last_frame = self.reconstruct_image(self.read_buffer)
    
    def set_tasks(self):
        self.write_task = Task()
        self.write_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/{self.ao_ch}", min_val=-10, max_val=10)
        self.write_task.timing.cfg_samp_clk_timing(
            rate=100,
            source="OnboardClock",
            active_edge=Edge.RISING,
            sample_mode=AcquisitionType.CONTINUOUS,
            # samps_per_chan=1000,
        )
        # self.write_task = Task()
        # self.write_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/ao0", min_val=-8, max_val=8)
        # self.write_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/ao3", min_val=-8, max_val=8)
        # self.write_task.timing.cfg_samp_clk_timing(
        #     rate=self.rate,
        #     source="OnboardClock",
        #     active_edge=Edge.RISING,
        #     sample_mode=AcquisitionType.CONTINUOUS,
        # )
        # self.write_task.out_stream.output_buf_size = self.total_px * 2 * self.buf_size
        # self.write_task.register_every_n_samples_transferred_from_buffer_event(self.total_px, self.write_callback)
        # self.writer = AnalogMultiChannelWriter(self.write_task.out_stream)
        # for _ in range(self.buf_size):
        #     self.writer.write_many_sample(self.write_signal)

        # self.read_task = Task()
        # self.read_task.ai_channels.add_ai_voltage_chan(f"{self.dev}/ai7", min_val=-5, max_val=5)
        # self.read_task.timing.cfg_samp_clk_timing(
        #     rate=self.rate,
        #     source="OnboardClock",
        #     active_edge=Edge.RISING,
        #     sample_mode=AcquisitionType.CONTINUOUS,
        # )
        # self.read_task.in_stream.input_buf_size = self.total_px * self.buf_size
        # self.read_task.triggers.start_trigger.cfg_dig_edge_start_trig(f"/{self.dev}/ao/StartTrigger", Edge.RISING)
        # self.read_task.register_every_n_samples_acquired_into_buffer_event(self.total_px, self.read_callback)
        # self.reader = AnalogMultiChannelReader(self.read_task.in_stream)

    # def write_callback(self, task_handle, every_n_samples_event_type, number_of_samples, callback_data):
    #     self.writer.write_many_sample(self.write_signal, timeout=1)
    #     return 0
    
    # def read_callback(self, task_handle, every_n_samples_event_type, number_of_samples, callback_data):
    #     self.reader.read_many_sample(self.read_buffer, number_of_samples, timeout=1)
    #     self.update(self.reconstruct_image(self.read_buffer))
    #     return 0
        
    def display_frame(self):
        print('Display func here')
        # # np.vstack([row for row in frame])
        self.viewer.setImage(
            self.frame_buffer.pop(0),
            autoHistogramRange=False,
        )


if __name__ == '__main__':
    app = QApplication(sys.argv)
    main = ConfocalMicroscopy()
    main.show()
    sys.exit(app.exec_())
